# **Quantum Schedulling para Processos Industriais** 

#### **Integrantes**
Gabriel Antunes, João Kruse

### **Introdução**
O mercado de produtos em geral tem tendido a uma mudança para um sistema com uma manutenção de estoques menores e buffers devido a alta competitividade. Na industria de alimentos e bebidas, isso se mostra mais extremo, com tendencias de mercado e demandas sendo voláteis e exigindo uma grande quantidade de produtos, o que exige uma produção flexível e ágil. No caso de cervejarias mais especificamente, temos ainda maiores dificuldades associadas ao fato de, além dos problemas de planejamento e agendamento das tarefas comuns, termos longos lead times devido a etapas como a fermentação. Isso faz com que metodologias para esse planejamento, agendamento e alocação de recursos escassos sejam extramamente necessárias[1]. Desse modo, nesse trabalho nós temos como objetivod utilizar computação quântica, mais especificamente QAOA, para a otimização desses processos.

### **Descrição do Problema**

Dentro de uma cervejaria, o processo de fabricação é composto por um conjunto de etapas, que podem ser esquematizadas como na figura abaixo.  

![](https://drive.google.com/uc?id=10GLFOYjt5uyzQu9AlKDD32D0fSi894pE)

Como podemos ver em [1], existem diversas etapas possíveis para o agendamente que exigem a solução de problemas de otimização NP-Difíceis. Dessa forma, muitas delas poderiam se beneficiar de algoritmos quânticos para a sua solução. Nesse trabalho, vamos focar específicamente no problema de Single Machine Scheduling (SMS), no qual temos um conjunto de tarefas com tempos de realização, custos e prazos associados e queremos encontrar a sequência delas segundo a qual temos o menor atraso somado.
Assim, temos a seguinte descrição para o nosso problema:  
  Dados W = {$w_1, w_2, w_3,...,w_n$} um conjunto com os diferentes tipos de cervejas a serem produzidos (tarefas), cada uma com um tempo de fermentação $t_i$, um peso $c_i$ e um prazo $p_i$ associado, queremos encontrar a sequência de realização das tarefas que resulte no menor atraso somado, assumindo que 2 tarefas não podem ser realizadas simultaneamente (utilizam uma mesma máquina). Isso corresponde a seguinte função de custo:  
  $$C = \sum_j c_j\sum_{(t_j-p_j)<t<h}x_{j,t}(t+t_j-p_j)$$
  onde $x_{j,t}$ é uma variável binária sendo 1 se o processo j começa no instante t e 0 caso contrário. Além disso, $h=\sum_j t_j-min\{t\}$, ou seja, o tempo total até o início da execução da última tarefa no mais tardar. 
  Por fim, temos como restrição desse problema que $\sum_t x_{j,t} = 1$, ou seja, que cada tarefa possui apenas um ponto inicial e que não haja sobreposição de tarefas [2]. O problema com os pesos é um problema NP-difícil (inclusive seu caso especial em que todos os produtos possuem o mesmo prazo corresponde ao problema da mochila, embora haja casos específicos para os quais existe uma solução polinomial usando Moore-Hodgson)[3], e o uso de QAOA pode trazer um benefício para sua solução ainda na era NISQ (outros algoritmos quânticos como quantum annealing também poderiam ser usados).

### **QAOA**
Existe mais de uma forma possível para a implementação desse problema. Uma das possibilidades é apresentada em [4] e outra ainda pode ser adaptada a partir do caso em [5], já que o problema SMS para o caso com prazos iguais pode ser convertido em um problema da mochila também. O exemplo em [4], que mapeia o problema como mostrado na figura abaixo. ![](https://drive.google.com/uc?id=1pW9lohoG0aB0MWV-TKEMN_5I0XcYzRRA)

### **Implementação**
Temos alguns exemplos de implementação mostrados abaixo. No primeiro, a função objetivo é obtida a partir do problema quadrático com suas restrições convertidas em penalidades. Como podemos ver, a solução obtida pelo QAOA corresponde a solução ideal. Na sequência, temos uma implementação seguindo o modelo apresentado em [4], e por fim uma implementação foi baseado no problema da mochila (Exercício 4) do IBM Quantum Challenge Fall 2021 (incompleta).

In [ ]:
!pip install qiskit
!pip install qiskit_optimization
!pip install pyquil
!pip install quantum-grove

In [2]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import LinearEqualityToPenalty
from qiskit import BasicAer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, SolutionSample, OptimizationResultStatus

Implementação 1

In [3]:
qubo = QuadraticProgram()
qubo.binary_var('x1')
qubo.binary_var('x2')
qubo.binary_var('x3')

qubo.binary_var('y1')
qubo.binary_var('y2')
qubo.binary_var('y3')

qubo.binary_var('z1')
qubo.binary_var('z2')
qubo.binary_var('z3')

peso    = [1,2,3]
duracao = [2,2,3]
prazo   = [4,4,4] 

qubo.minimize(quadratic={
  ('x1', 'y2'): (duracao[0]-prazo[1])*peso[1]+(duracao[0]+duracao[1]-prazo[2])*peso[2],
  ('x1', 'z2'): (duracao[0]-prazo[2])*peso[2]+(duracao[0]+duracao[2]-prazo[1])*peso[1],
  ('y1', 'x2'): (duracao[1]-prazo[0])*peso[0]+(duracao[1]+duracao[0]-prazo[2])*peso[2],
  ('y1', 'z2'): (duracao[1]-prazo[2])*peso[2]+(duracao[1]+duracao[2]-prazo[0])*peso[0],
  ('z1', 'x2'): (duracao[2]-prazo[0])*peso[0]+(duracao[2]+duracao[0]-prazo[1])*peso[1],
  ('z1', 'y2'): (duracao[2]-prazo[1])*peso[1]+(duracao[2]+duracao[1]-prazo[0])*peso[0],
  }
)
qubo.linear_constraint(linear={'x1': 1, 'x2': 1, 'x3': 1}, sense='==', rhs=1)
qubo.linear_constraint(linear={'y1': 1, 'y2': 1, 'y3': 1}, sense='==', rhs=1)
qubo.linear_constraint(linear={'z1': 1, 'z2': 1, 'z3': 1}, sense='==', rhs=1)
qubo.linear_constraint(linear={'x1': 1, 'y1': 1, 'z1': 1}, sense='==', rhs=1)
qubo.linear_constraint(linear={'x2': 1, 'y2': 1, 'z2': 1}, sense='==', rhs=1)
qubo.linear_constraint(linear={'x3': 1, 'y3': 1, 'z3': 1}, sense='==', rhs=1)

lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qubo)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 72 x1 - 72 x2 - 72 x3 - 72 y1 - 72 y2 - 72 y3 - 72 z1 - 72 z2 - 72 z3 +
      [ 72 x1^2 + 72 x1*x2 + 72 x1*x3 + 72 x1*y1 - 8 x1*y2 + 72 x1*z1 - 8 x1*z2
      + 72 x2^2 + 72 x2*x3 - 4 x2*y1 + 72 x2*y2 + 2 x2*z1 + 72 x2*z2 + 72 x3^2
      + 72 x3*y3 + 72 x3*z3 + 72 y1^2 + 72 y1*y2 + 72 y1*y3 + 72 y1*z1
      - 10 y1*z2 + 72 y2^2 + 72 y2*y3 - 2 y2*z1 + 72 y2*z2 + 72 y3^2 + 72 y3*z3
      + 72 z1^2 + 72 z1*z2 + 72 z1*z3 + 72 z2^2 + 72 z2*z3 + 72 z3^2 ]/2 + 108
Subject To

Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= y1 <= 1
 0 <= y2 <= 1
 0 <= y3 <= 1
 0 <= z1 <= 1
 0 <= z2 <= 1
 0 <= z3 <= 1

Binaries
 x1 x2 x3 y1 y2 y3 z1 z2 z3
End



In [4]:
algorithm_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

exact_result = exact.solve(qubo)
print(exact_result)

qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: -5.0
optimal value: [0. 0. 1. 1. 0. 0. 0. 1. 0.]
status: SUCCESS
optimal function value: -5.0
optimal value: [0. 0. 1. 1. 0. 0. 0. 1. 0.]
status: SUCCESS


Implementação 2

In [45]:
"""
!!!Precisa usar pyquil V2 ou menor!!!
Baseado em https://github.com/mstechly/grove/blob/master/grove/pyqaoa/tsp_qaoa.py
Finding a solution to the travelling salesman problem.
This code is based on the following project made by BOHR.TECHNOLOGY:
https://github.com/BOHRTECHNOLOGY/quantum_tsp
Which was in turn based on the articles by Stuart Hadfield:
https://arxiv.org/pdf/1709.03489.pdf
https://arxiv.org/pdf/1805.03265.pdf
"""
import pyquil.api as api
from grove.pyqaoa.qaoa import QAOA
import pyquil.latex
from grove.alpha.arbitrary_state.arbitrary_state import create_arbitrary_state
from pyquil.paulis import PauliTerm, PauliSum
from pyquil.quil import Program
from pyquil.gates import X
import scipy.optimize
import numpy as np


def solve_sms(num_tarefas, num_t, w, d, p, steps=3, ftol=1.0e-4, xtol=1.0e-4):
    connection = api.QVMConnection()
    list_of_qubits = list(range(num_tarefas*num_t))
    number_of_qubits = len(list_of_qubits)
    cost_operators = create_cost_hamiltonian(num_tarefas,num_t, w, d, p)
    driver_operators = create_mixer_operators(num_tarefas, num_t)
    initial_state_program = create_initial_state_program(num_tarefas, num_t)

    minimizer_kwargs = {'method': 'Nelder-Mead',
                            'options': {'ftol': ftol, 'xtol': xtol,
                                        'disp': print_fun}}

    vqe_option = {'disp': print_fun, 'return_all': True,
                  'samples': None}

    qaoa_inst = QAOA(connection, list_of_qubits, steps=steps, cost_ham=cost_operators,
                     ref_ham=driver_operators, driver_ref=initial_state_program, store_basis=True,
                     minimizer=scipy.optimize.minimize,
                     minimizer_kwargs=minimizer_kwargs,
                     vqe_options=vqe_option)
    
    betas, gammas = qaoa_inst.get_angles()
    most_frequent_string, _ = qaoa_inst.get_string(betas, gammas, samples=1000)
    solution = binary_state_to_points_order(most_frequent_string, num_t, num_tarefas)
    return solution


def create_cost_hamiltonian(num_tarefas, num_t, w, d, p):
    cost_operators = []
    for i in range(num_tarefas):
        for t in range(max(d[i]-p[i],0), num_t):
            qubit_1 = num_t*i + t
            cost_operators.append(PauliTerm("Z", qubit_1, w[i]*(t+d[i]-p[i])))
    cost_hamiltonian = [PauliSum(cost_operators)]
    return cost_hamiltonian


def create_mixer_operators(num_tarefas, num_t):
    """
    Creates mixer operators for the QAOA.
    It's based on equations 54 - 58 from https://arxiv.org/pdf/1709.03489.pdf
    Indexing here comes directly from section 4.1.2 from paper 1709.03489, equations 54 - 58.
    """
    n = num_t
    mixer_operators = []
    for i in range(n - 1):
        for u in range(num_tarefas):
            for v in range(num_tarefas):
                first_part = 1
                first_part *= s_plus(n, u, i)
                first_part *= s_plus(n, v, i+1)
                first_part *= s_minus(n, u, i+1)
                first_part *= s_minus(n, v, i)

                second_part = 1
                second_part *= s_minus(n, u, i)
                second_part *= s_minus(n, v, i+1)
                second_part *= s_plus(n, u, i+1)
                second_part *= s_plus(n, v, i)
                mixer_operators.append(first_part + second_part)
    return mixer_operators


def create_initial_state_program(num_tarefas, num_t):
    '''
    Cria um estado inicial em que cada tarefa possui apenas um tempo inicial
    '''
    initial_state = Program()
    for i in range(num_tarefas):
        for t in range(num_t):
            initial_state.inst(X(t*num_t + 2*i))

    return initial_state


def s_plus(i, j, t):
    qubit = t + i*j
    return PauliTerm("X", qubit) + PauliTerm("Y", qubit, 1j)


def s_minus(i, j, t):
    qubit = t + i*j
    return PauliTerm("X", qubit) - PauliTerm("Y", qubit, 1j)



def binary_state_to_points_order(binary_state,num_t, num_tarefas):
    """
    Transforms the the order of points from the binary representation: [1,0,0,0,1,0,0,0,1],
    to the binary one: [0, 1, 2]
    """
    print(binary_state)
    points_order = []
    number_of_points = num_t
    for p in range(num_tarefas):
        for j in range(num_t):
            if binary_state[num_t * p + j] == 1:
                points_order.append(j)
                break
    return points_order


def print_fun(x):
    print(x)


if __name__ == "__main__":
    num_tarefas = 2
    num_t = 4
    w = [3,2]
    p = [3,3]
    d = [2,2]
    solution = solve_sms(num_tarefas, num_t, w, d, p, steps=2, xtol=1, ftol=1)
    print("The most frequent solution")
    print(solution)

                     models will be ineffective
Optimization terminated successfully.
         Current function value: 10.000000
         Iterations: 1
         Function evaluations: 5
(1, 0, 1, 0, 1, 0, 1, 0)
The most frequent solution
[0, 0]


Implementação 3

In [ ]:
from typing import List, Union
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble
from qiskit.compiler import transpile
from qiskit.circuit import Gate
from qiskit.circuit.library.standard_gates import *
from qiskit.circuit.library import QFT

In [ ]:
# Código incompleto

def solver_function(L1: list, L2: list, C1: list, C2: list, C_max: int) -> QuantumCircuit:
    
    # the number of qubits representing answers
    index_qubits = len(L1)
    
    # the maximum possible total cost
    max_c = sum([max(l0, l1) for l0, l1 in zip(C1, C2)])
    
    # the number of qubits representing data values can be defined using the maximum possible total cost as follows:
    data_qubits = math.ceil(math.log(max_c, 2)) + 1 if not max_c & (max_c - 1) == 0 else math.ceil(math.log(max_c, 2)) + 2
    
    ### Phase Operator ###
    # return part
    def phase_return(index_qubits: int, gamma: float, L1: list, L2: list, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qr_index = QuantumRegister(index_qubits, "index")
        qc = QuantumCircuit(qr_index)

        for i in range(index_qubits):
            qc.u1(gamma * (L2[i] - L1[i]), i)

        return qc.to_gate(label=" phase return ") if to_gate else qc

    ##########################################################################
   
    # penalty part
    def subroutine_add_const(data_qubits: int, const: int, to_gate=True) -> Union[Gate, QuantumCircuit]:
        qc = QuantumCircuit(data_qubits)

        b = format(const,'0{}b'.format(data_qubits))[::-1]

        for idx, s in enumerate(reversed(range(data_qubits))):
            for i in range(idx+1):
                if(b[i] == 1):
                    qc.p(np.pi/(2**i),s)

        return qc.to_gate(label=" [+"+str(const)+"] ") if to_gate else qc

    ##########################################################################

    # penalty part
    def const_adder(data_qubits: int, const: int, to_gate=True) -> Union[Gate, QuantumCircuit]:

        qr_data = QuantumRegister(data_qubits, "data")
        qc = QuantumCircuit(qr_data)

        qc.append(QFT(data_qubits, do_swaps = False), qr_data)
        qc.append(subroutine_add_const(data_qubits, const), qr_data)
        qc.append(QFT(data_qubits, inverse=True, do_swaps = False), qr_data)
        qc = qc.decompose()
        
        return qc.to_gate(label=" [ +" + str(const) + "] ") if to_gate else qc

    ##########################################################################

    # penalty part
    def cost_calculation(index_qubits: int, data_qubits: int, list1: list, list2: list, to_gate = True) -> Union[Gate, QuantumCircuit]:

        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qc = QuantumCircuit(qr_index, qr_data)

        for i, (val1, val2) in enumerate(zip(list1, list2)):

            qc.append(const_adder(data_qubits, val1).control(), [qr_index[i]]+qr_data[:])
            qc.x(qr_index[i])
            qc.append(const_adder(data_qubits, val2).control(), [qr_index[i]]+qr_data[:])       
            qc.x(qr_index[i])

        return qc.to_gate(label=" Cost Calculation ") if to_gate else qc
    
    ##########################################################################

    # penalty part
    def constraint_testing(data_qubits: int, C_max: int, to_gate = True) -> Union[Gate, QuantumCircuit]:

        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_data, qr_f)

        for i in range(data_qubits):
            qc.x(i)

        qc.mcx(qr_data, qr_f)

        for i in range(data_qubits):
            qc.x(i)

        return qc.to_gate(label=" Constraint Testing ") if to_gate else qc

    ##########################################################################

    def penalty_dephasing(data_qubits: int, alpha: float, gamma: float, to_gate = True) -> Union[Gate, QuantumCircuit]:

        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_data, qr_f)

        for i in range(data_qubits):
            qc.cu1(2**i*alpha*gamma, qr_f, i)

        qc.u1(-2**data_qubits*alpha*gamma, qr_f)

        return qc.to_gate(label=" Penalty Dephasing ") if to_gate else qc
  
    ##########################################################################

    # penalty part
    def reinitialization(index_qubits: int, data_qubits: int, C1: list, C2: list, C_max: int, to_gate = True) -> Union[Gate, QuantumCircuit]:

        qr_index = QuantumRegister(index_qubits, "index")
        qr_data = QuantumRegister(data_qubits, "data")
        qr_f = QuantumRegister(1, "flag")
        qc = QuantumCircuit(qr_index, qr_data, qr_f)

        qc.append(constraint_testing(data_qubits, C_max).inverse(), qr_data[:] + qr_f[:])
        qc.append(cost_calculation(index_qubits, data_qubits, C1, C2).inverse(), qr_index[:] + qr_data[:])
        qc.append(phase_return(index_qubits, gamma, L1, L2).inverse(), qr_index)

        return qc.to_gate(label=" Reinitialization ") if to_gate else qc

    ##########################################################################
    
    ### Mixing Operator ###
    def mixing_operator(index_qubits: int, beta: float, to_gate = True) -> Union[Gate, QuantumCircuit]:

        qr_index = QuantumRegister(index_qubits, "index")
        qc = QuantumCircuit(qr_index)

        for i in range(index_qubits):  
            qc.rx(beta, i)

        return qc.to_gate(label=" Mixing Operator ") if to_gate else qc
    
    ##########################################################
    
    qr_index = QuantumRegister(index_qubits, "index") # index register
    qr_data = QuantumRegister(data_qubits, "data") # data register
    qr_f = QuantumRegister(1, "flag") # flag register
    cr_index = ClassicalRegister(index_qubits, "c_index") # classical register storing the measurement result of index register
    qc = QuantumCircuit(qr_index, qr_data, qr_f, cr_index)
    
    ### initialize the index register with uniform superposition state ###
    qc.h(qr_index)
    
    p = 5
    alpha = 1
    for i in range(p):
        
        ### set fixed parameters for each round ###
        beta = 1 - (i + 1) / p
        gamma = (i + 1) / p
        
        ### return part ###
        qc.append(phase_return(index_qubits, gamma, L1, L2), qr_index)
        
        ### step 1: cost calculation ###
        qc.append(cost_calculation(index_qubits, data_qubits, C1, C2), qr_index[:] + qr_data[:])
        
        ### step 2: Constraint testing ###
        qc.append(constraint_testing(data_qubits, C_max), qr_data[:] + qr_f[:])
        
        ### step 3: penalty dephasing ###
        qc.append(penalty_dephasing(data_qubits, alpha, gamma), qr_data[:] + qr_f[:])
        
        ### step 4: reinitialization ###
        qc.append(reinitialization(index_qubits, data_qubits, C1, C2, C_max), qr_index[:] + qr_data[:] + qr_f[:])
        
        ### mixing operator ###
        qc.append(mixing_operator(index_qubits, beta), qr_index)

    ### measure the index ###
    ### since the default measurement outcome is shown in big endian, it is necessary to reverse the classical bits in order to unify the endian ###
    qc.measure(qr_index, cr_index[::-1])
    
    return qc

### Referências

[1] **"Optimal production planning and scheduling in breweries"**, Georgiadis, G et al, 2021  
[2] **"Quantum Approximate Optimization with Hard and Soft Constraints"**, Hadfield, S. et. al, 2017  
[3] **"Knapsack-Like Scheduling Problems, the Moore-Hodgson Algorithm and the ‘Tower of Sets’ Property "**, Lawler, E, 1994  
[4] **"From the Quantum Approximate Optimization Algorithm to a Quantum Alternating Operator Ansatz"**, Hadfield, S. et. al, 2019  
[5] **"Knapsack Problem variants of QAOA for battery revenue optimisation"**, P. Dupuy de la Grand’rive. et. al, 2019